In [ ]:
from IPython.core.display import HTML
css = open('../data/style-table.css').read() + open('../data/style-notebook.css').read()
HTML('../data/<style>{}</style>'.format(css))

In [ ]:
%config IPCompleter.greedy = True

In [ ]:
from thetradeobject import SumReqFields, TheTradeObject
from tradestyle import TradeFormat

In [ ]:
import datetime, os


from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

from journalfiles import JournalFiles
import pandas as pd
from structjour.pandasutil import DataFrameUtil, InputDataFrame, ToCSV_Ticket as Ticket
from structjour.tradeutil import ReqCol, FinReqCol, XLImage, TradeUtil
#TradeUtil, FinReqCol, ReqCol, 

jf=JournalFiles(theDate=datetime.date(2018, 9, 21),infile = 'trades.csv', outdir = 'out/', mydevel=True)
jf._printValues()
        
tkt = Ticket(jf)

tu = TradeUtil()
trades, jf =tkt.newDFSingleTxPerTicket()
# trades = pd.read_csv(jf.inpathfile)

idf = InputDataFrame()
reqCol = ReqCol()
finalReqCol = FinReqCol()

In [ ]:
DataFrameUtil.checkRequiredInputFields(trades, reqCol.columns)
trades = idf.zeroPadTimeStr(trades)
trades = trades.sort_values([reqCol.acct, reqCol.ticker, reqCol.time])
trades = idf.mkShortsNegative(trades)

In [ ]:
swingTrade = idf.getOvernightTrades(trades)

In [ ]:
swingTrade = idf.figureOvernightTransactions(trades)
trades = idf.insertOvernightRow(trades,swingTrade)

In [ ]:
trades = tu.addFinReqCol(trades)
newTrades = trades[finalReqCol.columns]
newTrades.copy()
nt = newTrades.sort_values([finalReqCol.ticker,finalReqCol.acct,  finalReqCol.time])

In [ ]:
nt.head()

In [ ]:
nt = tu.writeShareBalance(nt)
nt.head()

In [ ]:
nt = tu.addStartTime(nt)
nt.head()

In [ ]:
nt = nt.sort_values([finalReqCol.start, finalReqCol.acct, finalReqCol.time])
nt.head()

In [ ]:
nt = tu.addTradeIndex(nt)
nt.head()


In [ ]:
nt = tu.addTradePL(nt)
nt.head()

In [ ]:
nt = tu.addTradeDuration(nt)
nt.head()

In [ ]:
nt = tu.addTradeName(nt)
nt.head()

In [ ]:
nt=DataFrameUtil.addRows(nt,1)
nt.head()

In [ ]:
nt = tu.addSummaryPL(nt)
nt.head()

In [ ]:
    
ldf=tu.getTradeList(nt)

In [ ]:
ldf[3]
df=ldf[3]

In [ ]:
dframe = DataFrameUtil.addRows(nt, 2)


### Skipping a bunch here- all extra(?)

In [ ]:
topMargin = 10
newdf = DataFrameUtil.createDf(dframe,  topMargin)
newdf

In [ ]:
insertsize = 25
dframe = newdf.append(dframe, ignore_index = True)
dframe

In [ ]:
#Add rows and append each trade, leaving space for an image. Create a list of names and rows numbers 
# to place images within the excel file.
# First, prepare the image names and the locations to place the images Save it in 
# imageLocation each entry [rowNum, shortName, name, timeOfTrade, durationOfTrade]
imageLocation = list()
response = input("Would you like to enter strategy names, targets and stops?")
interview = True if response.lower().startswith('y') else False
for tdf in ldf :
    imageName='{0}_{1}_{2}_{3}.jpeg'.format (tdf[finalReqCol.tix].unique()[-1].replace(' ',''), 
           tdf[finalReqCol.name].unique()[-1].replace(' ','-'),
           tdf[finalReqCol.start].unique()[-1],
           tdf[finalReqCol.dur].unique()[-1])

    # TODO handle empty string in the tdf
    imageLocation.append([len(tdf) + len(dframe) + 2, 
                          tdf.Tindex.unique()[0].replace(' ', '') + '.jpeg',
                          imageName,
                          tdf.Start.unique()[-1],
                        tdf.Duration.unique()[-1]])
    print(len(tdf) + len(dframe) + 2)

    dframe = dframe.append(tdf, ignore_index = True)
    dframe = DataFrameUtil.addRows(dframe, insertsize)
    print(len(dframe))

In [ ]:
imageLocation

In [ ]:
nt = dframe


wb = Workbook()
ws = wb.active
for r in dataframe_to_rows(nt, index=False, header=False):
    ws.append(r)

for name, cell  in zip(nt.columns, ws[topMargin]) :
    cell.value = name
    
XL = XLImage()
         

In [ ]:
srf = SumReqFields()
tradeSummaries = list()
tf = TradeFormat(wb)
assert (len(ldf) == len(imageLocation))
     

In [ ]:
for loc, tdf in zip(imageLocation, ldf) :
#     print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(loc[1], loc[2], loc[3]))
    img = XL.getAndResizeImage(loc[2], jf.outdir)
    cellname = 'J' + str(loc[0])
    ws.add_image(img, cellname)
    
    #Put together the summary info and interview the trader
    tto=TheTradeObject(tdf, interview)
    tto.runSummary()
    tradeSummaries.append(tto)
    
    #Place the format shapes/styles in the worksheet
    tf.formatTrade(ws, anchor=(1, loc[0]))

In [ ]:
# srf = SumReqFields()
# tradeSummaries = list()
# assert (len(ldf) == len(imageLocation))

In [ ]:
# for loc, tdf in zip(imageLocation, ldf) :
#     print (loc)
# #     print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(loc[1], loc[2], loc[3]))
#     img = XL.getAndResizeImage(loc[2], jf.outdir)
#     cellname = 'J' + str(loc[0])
#     ws.add_image(img, cellname)
#     tto=TheTradeObject(df)
#     tto.runSummary()
#     tradeSummaries.append(tto)
#     tf.formatTrade(ws, anchor=(1, loc[0]))
# print("Done with interview")
    

In [ ]:
# tradeSummaries[0].TheTrade['Entry1']

In [ ]:
jf.mkOutdir() 
saveName=jf.outpathfile
count=1
while True :
    try :
        wb.save(saveName)
    except PermissionError as ex :
        print(ex)
        print("Failed to create file {0}.{1}".format(saveName, ex))
        print("Images from the clipboard were saved  in {0}".format(jf.outdir))
        (nm, ext) = os.path.splitext(jf.outpathfile)
        saveName = "{0}({1}){2}".format(nm,count,ext)
        print("Will try to save as {0}".format(saveName))
        count=count+1
        if count==6:
            print("Giving up. PermissionError")
            raise (PermissionError("Failed to create file {0}".format(saveName)))
        continue
    except Exception as ex:
        print (ex)
    break
print("Done!")

In [ ]:
srf.columns



### Note that the interview stuff will be taken care of during the image insertion allowing the user to do all the review of a tradeafter viewing the chart. The interview stuff will include: 1. strategy  2. brief strategy note  3. target  4. stop loss  5. description of trade.  6. Notes and analysis of the trade

### On Second thought, the strategy note, trade explanation and analysis will be left to do in excel. The Gui Version os structjour will implement the review. Otherwise I would have to include some facility for entering multiple sentences in the console app.  Not that difficult but not that helpful when it can be done in Excel wilh all the visuals present.